# Задача 1

Найдем точечную оценку с помощью второго момента методом моментов
\begin{align*}
    \mathbb{E}_{\lambda}\left(X_1^2\right) = \overline{X^2} &\Leftrightarrow \\
    (16\lambda^{*})^2 + 16\lambda^{*} - \overline{X^2} = 0 &\Leftrightarrow \\
    \lambda^{*} = \frac{\sqrt{4 * \overline{X^2} + 1} - 1}{32}.
\end{align*}
Осталось лишь использовать ее для оценки истинного параметра $\lambda$.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import chi2

chat_id = 846523402 # Ваш chat ID, не меняйте название переменной

In [2]:
def solution(x: np.array) -> float:
    # Измените код этой функции
    # Это будет вашим решением
    # Не меняйте название функции и её аргументы
    return (np.sqrt(4 * (x ** 2).mean() + 1) - 1) / (2 * 16) # Ваш ответ

def experiment(N=1000, n=10, mu=10) -> float:
    from scipy.stats import poisson
    import numpy as np

    total = []
    for i in range(N):
        total.append((mu - solution(poisson.rvs(mu=16 * mu, size=n))) ** 2)
    total = np.array(total).mean()
    return total

In [3]:
experiment()

0.05992037525135366

# Задача 2

Итак, на вход нам мы ожидаем получить реализацию случайных величин:
\begin{align*}
    Z_i = \sqrt{X_i^2 + Y_i^2}, i=1,...,n,
\end{align*}
где $n$ - количество выпущенных Иваном стрел. Тогда заметим, что
\begin{align*}
    \sum\limits_{k=1}^{n} \frac{Z_k^2}{44\sigma^2} &= \sum\limits_{k=1}^{n} \left(\frac{X_i^2 + Y_i^2}{44\sigma^2}\right) \\
     &= \sum\limits_{k=1}^{n} \left(\left(\frac{X_i}{\sqrt{44\sigma^2}}\right)^2 + \left(\frac{Y_i}{\sqrt{44\sigma^2}}\right)^2\right) = \eta \sim {\cal{X}}_{2n}^2,
\end{align*}
т.е получили случайную величину, которая имеет распределение хи-квадрат с $2n$ степенями свободы (это распределение известное и уже не зависит от параметра!) Поэтому можно построить **точный** доверительный интервал уровня $1 - \alpha$ для неизвестного стандартного отклонения $\sigma$. Так как
\begin{align*}
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < \eta < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < n \cdot \frac{\overline{Z^2}}{44\sigma^2} < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2} < \sigma^2 < \frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}} < \sigma < \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}\right)  = 1 - \alpha,
\end{align*}
где $x_{1 - \alpha / 2, 2n}^2$ и $x_{\alpha / 2, 2n}^2$ квантили распределения хи-квадрат с $2n$ степенями свободы уровней $1 - \alpha / 2$ и $\alpha / 2$ соответственно, а также
\begin{align*}
    \overline{Z^2} = \frac{1}{n} \cdot \sum\limits_{k=1}^{n} Z_k^2.
\end{align*}
Поэтому доверительный интервал для $\sigma$ при заданном уровне значимости $1 - \alpha$ имеет вид:
\begin{align*}
    \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}}, \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}
\end{align*}

In [4]:
def solution(p: float, x: np.array) -> tuple:
    alpha = 1 - p
    size = len(x)
    left = np.sqrt(size * (x ** 2).mean() / (44 * chi2.ppf(q=1 - alpha / 2, df=2 * size)))
    right = np.sqrt(size * (x ** 2).mean() / (44 * chi2.ppf(q=alpha / 2, df=2 * size)))
    
    return left, \
           right


def experiment(N: int, std: float) -> pd.DataFrame:
    std = std
    N = N

    alphas = [0.99, 0.9, 0.7, 0.9, 0.95, 0.9]
    sample_sizes = [1000, 1000, 100, 100, 10, 10]
    df = []

    data = zip(alphas, sample_sizes)

    for p, size in data:
        error_counter = 0
        ci_length = []
        
        for i in range(N):
            sample = np.sqrt(norm.rvs(loc=0, scale=np.sqrt(44 * std ** 2), size=size) ** 2 \
                + norm.rvs(loc=0, scale=np.sqrt(44 * std ** 2), size=size) ** 2)
            
            left, right = solution(p, sample)
            error_counter += not [left < std < right][0]
            ci_length.append(right - left)

        error_counter /= N
        ci_length = np.array(ci_length).mean()

        df.append([size, p, round(error_counter, 3), round(ci_length, 3)])
        
    df = pd.DataFrame(df, columns=['Размер выборки', 'Доверие', 'Частота ошибок', 'Длина интервала'])

    return df

In [5]:
experiment(5000, 1)

,Размер выборки,Доверие,Частота ошибок,Длина интервала
0,1000,0.99,0.010,0.082
1,1000,0.90,0.093,0.052
2,100,0.70,0.289,0.104
3,100,0.90,0.095,0.165
4,10,0.95,0.053,0.672
5,10,0.90,0.095,0.553
